<a href="https://colab.research.google.com/github/NoriaYousufi/Brain-Tumor-Detection/blob/main/Brain_Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import Libaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split

from keras.utils import normalize, to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

###Load the data

In [2]:
image_directory='/content/drive/MyDrive/Datasets'
no_tumor_images=os.listdir(image_directory+ '/no')
yes_tumor_images=os.listdir(image_directory+ '/yes')

print('No Tumor: ', len(no_tumor_images))
print('Tumor: ',len(yes_tumor_images))

No Tumor:  1500
Tumor:  1500


In [3]:
dataset = []
label = []
INPUT_SIZE = 64

In [4]:
# Seperate images
# No tumor
for i, image_name in enumerate(no_tumor_images):
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(image_directory, 'no', image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image

        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)  # Assuming 0 represents images without tumors


In [5]:
for i, image_name in enumerate(yes_tumor_images):
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(image_directory, 'yes', image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image

        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)  # Assuming 1 represents images with tumors


In [6]:
print(len(dataset))
print(len(label))

3000
3000


In [7]:
dataset = np.array(dataset)
label = np.array(label)

###Splitting training and test data

In [8]:
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [9]:
print(x_train.shape)
print(y_train.shape)

(2400, 64, 64, 3)
(2400,)


In [10]:
print(x_test.shape)
print(y_test.shape)

(600, 64, 64, 3)
(600,)


###Normalize data

In [11]:
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

###CNN model

In [12]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train,
    batch_size=16,
    verbose=1,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=False)

Epoch 1/10
150/150 [==============================] - 17s 103ms/step - loss: 0.5561 - accuracy: 0.7233 - val_loss: 0.4376 - val_accuracy: 0.8317
Epoch 2/10
150/150 [==============================] - 15s 98ms/step - loss: 0.4140 - accuracy: 0.8221 - val_loss: 0.3351 - val_accuracy: 0.8533
Epoch 3/10
150/150 [==============================] - 15s 101ms/step - loss: 0.2940 - accuracy: 0.8763 - val_loss: 0.2787 - val_accuracy: 0.8850
Epoch 4/10
150/150 [==============================] - 15s 101ms/step - loss: 0.2238 - accuracy: 0.9208 - val_loss: 0.2095 - val_accuracy: 0.9300
Epoch 5/10
150/150 [==============================] - 26s 173ms/step - loss: 0.1530 - accuracy: 0.9525 - val_loss: 0.1558 - val_accuracy: 0.9483
Epoch 6/10
150/150 [==============================] - 15s 102ms/step - loss: 0.1027 - accuracy: 0.9650 - val_loss: 0.1579 - val_accuracy: 0.9433
Epoch 7/10
150/150 [==============================] - 15s 101ms/step - loss: 0.0777 - accuracy: 0.9771 - val_loss: 0.1155 - val_acc

In [17]:
model.save('BrainTumor10Epochs.h5')

###Test on prediction set

In [29]:
import cv2
from keras.models import load_model
from PIL import Image
import numpy as np

In [30]:
model = load_model('/content/BrainTumor10Epochs.h5')

In [35]:
image = cv2.imread('/content/drive/MyDrive/Datasets/pred/pred2.jpg')
img = Image.fromarray(image)
img = img.resize((64,64))
img = np.array(img)
input_img = np.expand_dims(img, axis=0)
result = model.predict(input_img)
print(result)

1/1 [==============================] - 0s 24ms/step
[[0.]]
